In [ ]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
    json_dict = json.loads(json_string)
    return json_dict


def prettify_json(obj):
    pretty_json = json.dumps(obj, indent=2)
    return pretty_json


def write_to_json_file(obj, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(obj, json_file, indent=2)


In [ ]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [ ]:
def transform_graph(graph):
	nodes = { node['data']['id']: node['data'] for node in graph['elements']['nodes'] }
	edges = {}
	for edge in graph['elements']['edges']:
		if 'label' in edge['data']:
			label = edge['data']['label']
		else:
			label = ','.join(edge['data']['labels'])
			edge['data']['label'] = label
		
		if label not in edges:
			edges[label] = []
		edges[label].append(edge['data'])
	return (nodes, edges)
	

In [ ]:
config = read_ini_file('config.ini')
project_name = config['project']['name']
project_desc = config['project']['desc']
ifile = config['project']['ifile']
(project_name,project_desc,ifile)

# project_name = 'jpacman'
ifile = f'{project_name}-with-summaries.json'

In [ ]:
graph = read_json_file(ifile)
nodes,edges = transform_graph(graph)

list(nodes.keys()), list(edges.keys())

In [ ]:
def invert(edgeList):
    prefix = "inv_"
    return [{**edge,
            'source': edge['target'],
            'target': edge['source'],
            'label': prefix + edge.get('label', 'edge'),
        } for edge in edgeList]

# invert(edges['contains'])

In [ ]:
def compose(l1, l2, newlabel=None):
    mapping = {
        edge['source']: {
            'target': edge['target'], 
            'label': edge.get('label','edge1'), 
            'weight': edge.get('properties', {}).get('weight', 1)
        } for edge in l2 }
    
    result = {}
    for edge in l1:
        s1 = edge['source']
        t1 = edge['target']
        label = edge.get('label', 'edge2')
        properties = edge.get('properties', {})
        mappingEntry = mapping.get(t1)

        if mappingEntry:
            newWeight = mappingEntry['weight'] * properties.get('weight', 1)
            key = (s1, mappingEntry['target'])
            if key not in result:
                result[key] = {
                    'source': s1,
                    'target': mappingEntry['target'],
                    'label': newlabel or label + "-" + mappingEntry['label'],
                    'properties': {'weight': newWeight},
                }
            else:
                result[key]['properties']['weight'] += newWeight
    return list(result.values())


contains = [
    {"source": "class A", "target": "method x"},
    {"source": "class A", "target": "method x2"},
    {"source": "class B", "target": "method y"}
]

invokes = [
    {"source": "method x", "target": "method y"},
    {"source": "method x2", "target": "method y"}
]

inverted_contains = invert(contains)

calls = compose(compose(contains, invokes), inverted_contains)

calls


In [ ]:
def get_all_labels(objects):
    labels = set()
    for obj in objects.values():
        if 'labels' in obj and isinstance(obj['labels'], list):
            labels.update(obj['labels'])
    return labels

get_all_labels(nodes)

In [ ]:
def get_edge_node_labels(edge, nodes):
    src_labels = nodes.get(edge['source'], {}).get('labels', [])
    tgt_labels = nodes.get(edge['target'], {}).get('labels', [])

    return [(src_label, tgt_label) for src_label in src_labels for tgt_label in tgt_labels]

get_edge_node_labels(edges['invokes'][0], nodes)

In [ ]:
def get_source_and_target_labels(edge_list, nodes):
    edge_node_labels = {label for edge in edge_list for label in get_edge_node_labels(edge, nodes)}

    return edge_node_labels

get_source_and_target_labels(edges['invokes'], nodes)

In [ ]:
def get_ontology(edges, nodes):
    return {label: get_source_and_target_labels(edge, nodes) for label, edge in edges.items()}

get_ontology(edges, nodes)

In [ ]:
def lift(rel1, rel2, newlabel=None):
    return compose(compose(rel1, rel2), invert(rel1), newlabel)

lift(contains, invokes)

In [ ]:
get_all_labels(nodes)

In [ ]:
get_source_and_target_labels(edges['invokes'], nodes)

In [ ]:
get_ontology(edges,nodes)

In [ ]:
calls = lift(edges['hasScript'], edges['invokes'], 'calls')
calls

In [ ]:
constructs = compose(edges['hasScript'], edges['instantiates'], 'constructs')
constructs

In [ ]:
holds = compose(edges['hasVariable'], edges['type'], "holds")
holds

In [ ]:
accepts = compose(edges['hasScript'], compose(edges['hasParameter'], edges['type']), "accepts")
accepts

In [ ]:
returns = compose(edges['hasScript'], edges['returnType'], "returns")
returns

## Fix for nested classes

To make our life easier, let's change an edge label: "contains" is now only for packages to other packages or classes, while classes to classes is changed into "nests".

In [ ]:
nested_classes_set = set(
    edge['target'] for edge in edges['contains']
    if 'Structure' in nodes.get(edge['source'], {}).get('labels', []))
nested_classes_set

In [ ]:
top_level_classes = [
    node_id for node_id, node in nodes.items()
    if 'Structure' in node.get('labels', []) and node_id not in nested_classes_set
]

top_level_classes_set = set(top_level_classes)

top_level_classes_set

In [ ]:
new_contains = [edge for edge in edges['contains'] if edge['source'] not in top_level_classes_set]
new_contains

In [ ]:
nests = edges['nests'] \
		if 'nests' in edges \
		else [dict(edge, label="nests") 
				for edge in edges['contains'] 
				if edge['source'] in top_level_classes_set]
nests

In [ ]:
depends_calls = [edge for edge in lift(new_contains, calls, "depends_calls") if edge['source'] != edge['target']]
depends_holds = [edge for edge in lift(new_contains, holds, "depends_holds") if edge['source'] != edge['target']]
depends_constructs = [edge for edge in lift(new_contains, constructs, "depends_constructs") if edge['source'] != edge['target']]
depends_accepts = [edge for edge in lift(new_contains, accepts, "depends_accepts") if edge['source'] != edge['target']]

depends = depends_calls + depends_holds + depends_constructs + depends_accepts
depends

In [ ]:
def filter_objects_by_labels(data, labels):
    filtered_data = {}
    for key, object in data.items():
        if any(label in labels for label in object['labels']):
            filtered_data[key] = object
    return filtered_data


In [ ]:
abstract_nodes = filter_objects_by_labels(nodes, ["Container", "Structure", "Primitive", "Problem"])
abstract_edges = {
    'contains': new_contains,
    'specializes': edges['specializes'],
    'nests': nests,
    'calls': calls,
    'constructs': constructs,
    'holds': holds,
    'accepts': accepts,
    'returns': returns,
}

abstract_nodes, abstract_edges

In [ ]:
abstract_graph = {
    'elements': {
        'nodes': [{'data':node} for node in abstract_nodes.values()],
        'edges': [{'data':edge} for edge_list in abstract_edges.values() for edge in edge_list],
    },
}

abstract_graph['elements']


In [ ]:
def get_edges_with_labels(nodes, edge_list, label):
    # Filter edge_list to include only edges where both source and target nodes have label "Container"
    filtered_edges = [edge for edge in edge_list if label in nodes[edge['source']].get('labels', []) and label in nodes[edge['target']].get('labels', [])]
    return filtered_edges


In [ ]:
get_edges_with_labels(nodes, edges['contains'], 'Container')

In [ ]:
pkg_nodes = filter_objects_by_labels(nodes, ["Container", "Problem"])
pkg_edges = {
    'contains': get_edges_with_labels(nodes, edges['contains'], "Container"),
    'depends': depends
}

pkg_edges


In [ ]:
pkg_graph = {
    "elements": {
        "nodes": [ {"data": node} for node in list(pkg_nodes.values()) ],
        "edges": [ {"data": edge} for edge in sum(list(pkg_edges.values()), []) ]
    }
}

pkg_graph["elements"]

In [ ]:
write_to_json_file(abstract_graph, f"{project_name}-abstract.json")

In [ ]:
write_to_json_file(pkg_graph, f"{project_name}-more-abstract.json")